In [1]:
import os
# os.environ["LABEL_STUDIO_LOCAL_FILES_DOCUMENT_ROOT"] = "/home/patel_zeel"
# os.environ["LABEL_STUDIO_LOCAL_FILES_DOCUMENT_ROOT"] = "/home/shataxi.dubey"

os.environ["LABEL_STUDIO_LOCAL_FILES_DOCUMENT_ROOT"] = '/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data'

import sys
import requests
from os.path import join, exists, basename
from glob import glob
from label_studio_sdk import Client
from copy import deepcopy

import torch
import numpy as np
from ultralytics.utils.ops import xyxyxyxy2xywhr, xywhr2xyxyxyxy
from ultralytics.utils.metrics import probiou
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import warnings

from joblib import Parallel, delayed

sys.path.append("/home/patel_zeel/kilns_neurips24/scripts")

In [2]:
from utils import yolo_obb_to_label_studio_json, label_map
# label_map

## Label Studio Initialize

In [3]:
LABEL_STUDIO_URL = 'http://10.0.62.168:8081'
# API_KEY = '182f2ac0975d5cb5661c4ddc1b4ddc34d21c1867'
API_KEY = '2557d98fa3032fc9a2eb033baf2775ebc2e1a0ef'

ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)

## Process the input data

In [4]:
# pred_txt_path = "/home/patel_zeel/kilns_neurips24/gujarat/labels"
# image_read_dir = "/home/patel_zeel/kilns_neurips24/gujarat/images"

# pred_txt_path = "/home/shataxi.dubey/shataxi_work/uplode_to_label_studio/lucknow_labels_with_cfcbk/labels"
# image_read_dir = "/home/shataxi.dubey/shataxi_work/uplode_to_label_studio/lucknow_labels_with_cfcbk/images"

image_read_dir =  "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/gms_data/delhi_ncr_data/images_main"
pred_txt_path = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/predict/trench_width_delhi_ncr_obb_v2_640_100/delhi_ncr_data/labels"

ls_project_name = f"trench_width_delhi_ncr_obb_v3"

pred_files = sorted(glob(join(pred_txt_path, "*.txt")))
pred_base_names = set([basename(f) for f in pred_files])

print(len(pred_files))

288


## Create Project

In [5]:
image_read_dir = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/gms_data/delhi_ncr_data/images_main"
# print(f"Expected root: {Path(root)}")
# print(f"Provided local_store_path: {Path(local_store_path)}")



In [7]:
projects = ls.list_projects()
for project in projects:
    print(project.title)
    if project.title == ls_project_name:
        print("Project found. Using it...")
        break
else:
    print("No project found. Creating...")
    project = ls.start_project(title=ls_project_name, label_config='''
<View>
  <Image name="image" value="$image" rotateControl="true" zoomControl="true" zoom="true"/>
  <RectangleLabels name="label" toName="image">
    
    
  <Label value="Inner_Box" background="#FFA39E"/><Label value="Outer_Box" background="#63c205"/></RectangleLabels>
</View>
    ''')
storages = project.get_import_storages()
print(storages)
if len(storages) == 0:
    print("No storage found. Creating...")
    print("Connecting local storage...")
    print(image_read_dir)
    image_read_dir="/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/gms_data/delhi_ncr_data/images_main"
    project.connect_local_import_storage(image_read_dir, regex_filter=".*png")
else:
    assert len(storages) == 1
    print("Storage found. Using it...")
storage = storages[0]
if storage['last_sync'] is None:
    print("Syncing storage...")
    project.sync_storage(storage['type'], storage['id'])
else:
    print("Storage already synced.")

print("Getting tasks...")
tasks = project.get_tasks()
len(tasks)


--------------------------------------------
Request URL: http://10.0.62.168:8081/api/storages/localfiles?project=21
Response status code: 400
Response content:
{
  "id": "f6cca913-34f8-4db7-9458-1336abb97114",
  "status_code": 400,
  "version": "1.15.0",
  "detail": "Validation error",
  "exc_info": null,
  "validation_errors": {
    "non_field_errors": [
      "[ErrorDetail(string=\"Serving local files can be dangerous, so it's disabled by default. You can enable it with LOCAL_FILES_SERVING_ENABLED environment variable, please check docs: https://labelstud.io/guide/storage.html#Local-storage\", code='invalid')]"
    ]
  }
}

SDK error traceback:


trench_width_delhi_ncr_obb_v3
Project found. Using it...
[]
No storage found. Creating...
Connecting local storage...
/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/gms_data/delhi_ncr_data/images_main/


HTTPError: 400 Client Error: Bad Request for url: http://10.0.62.168:8081/api/storages/localfiles?project=21

In [34]:
projects = ls.list_projects()
for project in projects:
    print(project.title)
    if project.title == ls_project_name:
        print("Project found. Using it...")
        break
else:
    print("No project found. Creating...")
    project = ls.start_project(title=ls_project_name, label_config='''
<View>
  <Image name="image" value="$image" rotateControl="true" zoomControl="true" zoom="true"/>
  <RectangleLabels name="label" toName="image">
    
    
  <Label value="Inner_Box" background="#FFA39E"/><Label value="Outer_Box" background="#63c205"/></RectangleLabels>
</View>
    ''')
storages = project.get_import_storages()
print(storages)

if len(storages) == 0:
    print("No storage found. Creating...")
    print("Connecting local storage...")
    print(image_read_dir)
    project.connect_local_import_storage(image_read_dir, regex_filter=".*png")
    # Re-fetch storages after creating one
    storages = project.get_import_storages()
else:
    assert len(storages) == 1
    print("Storage found. Using it...")

# Ensure storages is not empty before accessing
if len(storages) > 0:
    storage = storages[0]
    if storage['last_sync'] is None:
        print("Syncing storage...")
        project.sync_storage(storage['type'], storage['id'])
    else:
        print("Storage already synced.")
else:
    print("Storage creation failed or not available.")

print("Getting tasks...")
tasks = project.get_tasks()
print(f"Number of tasks: {len(tasks)}")


trench_width_delhi_ncr_obb_v2
trench_width_delhi_ncr_obb
Project found. Using it...
[{'id': 13, 'type': 'localfiles', 'synchronizable': True, 'path': '/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/gms_data/delhi_ncr_data/images_main', 'regex_filter': '.*png', 'use_blob_urls': True, 'last_sync': '2025-01-22T07:05:06.161739Z', 'last_sync_count': 294, 'last_sync_job': None, 'status': 'completed', 'traceback': None, 'meta': {'attempts': 1, 'time_queued': '2025-01-22 07:05:04.459038+00:00', 'time_in_progress': '2025-01-22 07:05:04.463547+00:00', 'time_last_ping': '2025-01-22 07:05:04.463547+00:00', 'time_completed': '2025-01-22 07:05:06.161742+00:00', 'duration': 1.698195, 'tasks_existed': 0}, 'title': '', 'description': '', 'created_at': '2025-01-22T07:00:54.027018Z', 'project': 19}]
Storage found. Using it...
Storage already synced.
Getting tasks...
Number of tasks: 294


In [35]:
tasks[0]

{'id': 2092,
 'predictions': [{'id': 688,
   'result': [{'type': 'rectanglelabels',
     'from_name': 'label',
     'to_name': 'image',
     'original_width': 1120,
     'original_height': 1220,
     'image_rotation': 0,
     'value': {'rotation': 26.83956336975098,
      'x': 15.418297309972317,
      'y': 27.04560176307251,
      'width': 81.22051358222961,
      'height': 24.669376015663147,
      'rectanglelabels': ['Zigzag']}},
    {'type': 'rectanglelabels',
     'from_name': 'label',
     'to_name': 'image',
     'original_width': 1120,
     'original_height': 1220,
     'image_rotation': 0,
     'value': {'rotation': 25.871191024780277,
      'x': 19.132898094899648,
      'y': 38.77640311611417,
      'width': 63.456809520721436,
      'height': 8.289225399494171,
      'rectanglelabels': ['FCBK']}}],
   'model_version': 'v1',
   'created_ago': '3\xa0minutes',
   'score': 0.0,
   'cluster': None,
   'neighbors': None,
   'mislabeling': 0.0,
   'created_at': '2025-01-27T13:39:2

In [37]:
# ## Delete all predictions before going ahead
# os.system(f"curl -H 'Authorization: Token {API_KEY}' -X POST \"localhost:8082/api/dm/actions?id=delete_tasks_predictions&project={project.id}\"")

# for task in tqdm(tasks):
#     # part = tile.split("/")[-3]
#     # name = tile.split("/")[-1].replace(".txt", ".png")
#     # image_path = join(save_dir, name)
#     image_path = task['storage_filename']
#     txt_name = basename(image_path).replace(".png", ".txt")
    
#     preds = np.loadtxt(join(pred_txt_path, txt_name))
#     preds = np.atleast_2d(preds)

#     results = []
#     for pred in preds:
#         print(pred.shape)
#         label_id = int(pred[0])
#         x1, y1, x2, y2, x3, y3, x4, y4 = pred[1:-1]
#         score = pred[-1]
#         # if score > score_threshold:
#         prediction = yolo_obb_to_label_studio_json(label_id, x1, y1, x2, y2, x3, y3, x4, y4, score)
#           # predictions.append(prediction)
#         results.append(prediction['result'][0])
#     if len(results) > 0:
#       # I have tried parallelizing this but it doesn't work
#       project.create_prediction(task['id'], results, model_version="v1")

In [38]:
import os
import numpy as np
from tqdm import tqdm
from os.path import join, basename, exists

# Deleting all predictions before proceeding
os.system(f"curl -H 'Authorization: Token {API_KEY}' -X POST \"localhost:8082/api/dm/actions?id=delete_tasks_predictions&project={project.id}\"")

for task in tqdm(tasks):
    image_path = task['storage_filename']
    txt_name = basename(image_path).replace(".png", ".txt")
    txt_path = join(pred_txt_path, txt_name)
    
    # Check if the .txt file exists
    if not exists(txt_path):
        print(f"Skipping {txt_name}: corresponding .txt file not found.")
        continue

    # Load predictions from the .txt file
    preds = np.loadtxt(txt_path)
    preds = np.atleast_2d(preds)

    results = []
    for pred in preds:
        print(pred.shape)
        label_id = int(pred[0])
        print(pred[1:9])

        x1, y1, x2, y2, x3, y3, x4, y4 = pred[1:9]
        score = pred[-1]
    #     # Convert YOLO OBB to Label Studio format
        prediction = yolo_obb_to_label_studio_json(label_id, x1, y1, x2, y2, x3, y3, x4, y4, score)
        results.append(prediction['result'][0])

    if len(results) > 0:
    #     # Create prediction in Label Studio
        project.create_prediction(task['id'], results, model_version="v1")


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (7) Failed to connect to localhost port 8082 after 0 ms: Couldn't connect to server
  1%|          | 2/294 [00:00<00:17, 16.63it/s]

(10,)
[    0.76751     0.85728      0.8338     0.61435     0.15418     0.27046    0.087894     0.51339]
(10,)
[    0.72613     0.73924     0.74751     0.65748     0.19133     0.38776     0.16995     0.46952]
(10,)
[    0.82493     0.76276     0.84425     0.27341    0.064736     0.13327    0.045423     0.62262]
(10,)
[    0.81518     0.57126     0.81978     0.41074    0.096095     0.31635    0.091499     0.47687]
(10,)
[    0.65912     0.72274     0.68543     0.64958     0.16818     0.31616     0.14188     0.38932]
(10,)
[    0.65195     0.82965     0.74468     0.58424     0.17749     0.20001    0.084763     0.44542]


  1%|▏         | 4/294 [00:00<00:21, 13.49it/s]

(10,)
[    0.75694     0.84502     0.86293     0.58928     0.26568     0.20076     0.15968      0.4565]
(10,)
[    0.75921     0.73225     0.78933     0.65764     0.26222     0.32359      0.2321     0.39819]
(10,)
[    0.71787     0.88677     0.85356     0.67594     0.24231     0.16177     0.10662     0.37259]
(10,)
[    0.75358     0.78803     0.79114     0.72672     0.19917     0.25269      0.1616       0.314]
(10,)
[    0.17806      0.7015     0.37638     0.82206     0.80604     0.31055     0.60772     0.18999]
(10,)
[    0.29815      0.7079     0.35654     0.74123     0.71132     0.29138     0.65293     0.25805]


  3%|▎         | 8/294 [00:00<00:23, 12.26it/s]

...

(10,)
[      0.116     0.78157     0.44324     0.86937     0.80055     0.21431     0.47331     0.12651]
(10,)
[    0.26548     0.79755      0.3553     0.81905     0.65389     0.20547     0.56408     0.18397]
(10,)
[    0.84958     0.93746     0.90263     0.40965     0.11732    0.068607     0.06426     0.59642]
(10,)
[    0.82217     0.66255     0.82907     0.53213     0.13525     0.37342     0.12834     0.50384]
(10,)
[    0.20163     0.81329     0.53988     0.89401     0.86765     0.24421     0.52939      0.1635]
(10,)
[    0.33994     0.82026     0.44128     0.84297     0.73069     0.23204     0.62935     0.20933]


 99%|█████████▊| 290/294 [00:24<00:00, 12.61it/s]

(10,)
[     0.8167     0.78111     0.85209     0.49919     0.17216     0.28376     0.13677     0.56568]
(10,)
[    0.74979     0.80921     0.80951     0.51858     0.19441     0.25578     0.13468     0.54641]
(10,)
[     0.7204     0.69584     0.73859      0.6091     0.20659     0.37719      0.1884     0.46394]
(10,)
[    0.11116     0.71828     0.40647     0.80122     0.77226     0.13612     0.47694    0.053179]
(10,)
[    0.23145      0.7119     0.32114     0.73671     0.64019     0.14765      0.5505     0.12284]


100%|██████████| 294/294 [00:24<00:00, 12.06it/s]

(10,)
[    0.10422     0.81427     0.65668     0.85601     0.79029     0.30069     0.23783     0.25895]
(10,)
[    0.29389     0.84134     0.52009     0.85278     0.60998     0.29471     0.38378     0.28327]
(10,)
[    0.19741     0.79526     0.67824     0.82826     0.78949     0.21898     0.30866     0.18598]
(10,)
[    0.38505     0.79388       0.519      0.8005     0.59931      0.1894     0.46535     0.18279]


## Appendix

In [ ]:
# areas = []
# for item in json_list:
#     for pred in item["predictions"]:
#         area = pred['result'][0]['value']['width'] * pred['result'][0]['value']['height']
#         area = area * (1120 * 1120) / (100 * 100)
#         areas.append(area)
# len(areas)

In [ ]:
# max(areas), min(areas), np.mean(areas), np.median(areas)

In [ ]:
# import seaborn as sns

# sns.kdeplot(areas)

In [ ]:
# (areas < np.mean(areas) - 2 * np.std(areas)).sum()